#### Introduction

This notebook is intended for ArcGIS Enterprise administrators who are looking to create basemaps compatible with offline map areas in their portal. ArcGIS Online offers a variety of tile and vector basemaps that can be exported for this purpose. To effectively use these basemaps with offline map area-compatible applications, such as Field Maps, it is essential that each basemap includes embedded credentials for ArcGIS Online. These credentials are necessary to facilitate the creation of offline map areas. Both vector and tile services are supported in this process.

##### Requirements:
- ArcGIS account credentials (ideally viewer account)
- ArcGIS Online basemap service will need to support export (see list below)
- ArcGIS Enterprise account that has permissions to publish items
- ArcGIS Enterprise 10.9.1 and above are supported
- Tested with Python 3.9.2 / ArcGIS 2.3.1

#### Import libraries

In [1]:
import ipywidgets as widgets
from arcgis.gis import GIS

#### Choose what basemaps you want to include. Add or comment out the maps you want to include in processing.  

*Note: in order for basemaps to support offline areas they will need to have exporting enabled.*

ArcGIS Online Basemaps for export:
These maps are designed to support exporting small volumes of basemap tiles for offline use - not intended to be used to display live map tiles
- Vector Basemaps: https://www.arcgis.com/home/group.html?sortField=modified&sortOrder=asc&id=c61ab1493fff4b84b53705184876c9b0#content
- Tiled Basemaps: https://www.arcgis.com/home/group.html?sortField=title&sortOrder=asc&id=3a890be7a4b046c7840dc4a0446c5b31#content

In [ ]:

# folder on enterprise in which items will be created
basemap_folder = "AGOL Offline Basemaps"

# Specify the item IDs of the vector basemap you want to clone
# These maps below are designed to support exporting small volumes of basemap tiles for offline use 
# not intended to be used to display live map tiles
agol_item_id_list = [   
                        # tiled basemaps - deprecated basemaps not included
                        "9e42e0d4acde413a9a9eb5f05fe0a2e6", # World Hillshade (Dark) (for Export) 
                        "babedc22ebd64a428b77f7119c2591c3", # World Hillshade (for Export)
                        "226d23f076da478bba4589e7eae95952", # World Imagery (for Export)  
                        "5d85d897aee241f884158aa514954443", # World Ocean Base (for Export) 

                        # vector basemaps
                        "38649a45a3544c0e809d00ea86be78e6", # World Navigation Map (Dark - for Export) 
                        "4faaa4931a3541e5b7461c732a7bda1c", # World Ocean Reference (for Export)
                        "7f5fe58ee3c046da8d83980b7262b7f6", # World Street Map (for Export)
                        "758db17cc1ee4181a049d1fa5d0c6bf0", # World Street Map (with Relief - for Export)
                        "4b5200491af84f898e1e6aa494ed79e2", # World Navigation Map (for Export)
                        "e7817b59ec4b40e4a1e55241b8695534", # Hybrid Reference Layer (for Export)
                        "df541726b3df4c0caf99255bb1be4c86", # World Topographic Map (for Export)
                        "f37f80e4e53c4c25bbec2eab05371f62", # World Terrain with Labels (for Export) 
                        "1f472880dccc4e99a47e4745908165b2", # World Terrain Reference (for Export)
                        "f29b05507f594d00a916b33ebb8404f3", # World Terrain Base (for Export)
                        "8e848d9302e84dcba0de7aff8ac429dc", # World Street Map (Night - for Export)

                        #"16024e0cee3949fa89a17f483b7189a9", # National Geographic Style (for Export) 
                        # "a2824f0bd9724eb9882eef0d059581d1", # Light Gray Canvas (for Export)
                        # "d23123ae88a14088843bc552ad3e9868", # Light Gray Canvas Base (for Export) 
                        # "8e7636df0c7a4aafa4fe678081dd0d56", # Light Gray Canvas Reference (for Export) 
                        # "aa80feb1dfb946a1b0859df4acca14b1", # Dark Gray Canvas Reference (for Export)  
                        # "3175cf5b9e9f47f284c7b7d3c8d5b387", # Dark Gray Canvas Base (for Export)
                        # "e945c4f09c4345ffb9ae6761cedf5c72", # Dark Gray Canvas (for Export) 
                ]  


#### Provide ArcGIS Credentials and Connect to Enterprise GIS using form

This form is optional - there are a variety of secure methods for providing credentials - please see links below:

- For information on different authentication schemes see:
    - https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/
- For protecting your credentials see:
    - https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#protecting-your-credentials

In [ ]:
# created a class to be able to access form input from outside the on click event - perhaps a better way of doing this :) 
class UserInputForm:
    def __init__(self):

        # create form input elements
        self.form_agol_username = widgets.Text()
        self.form_agol_pw = widgets.Password()
        self.form_portal_url = widgets.Text(placeholder="https://exampleportal/portal")
        self.form_portal_username = widgets.Text()
        self.form_portal_pw = widgets.Password()
        self.form_connect_button = widgets.Button(description="Submit GIS Details")


        self.form_connect_button.on_click(self.submit_gis_details_clicked)

        self.form_items = [
            widgets.Box([widgets.Label(value="ArcGIS Online Username:"), self.form_agol_username]),
            widgets.Box([widgets.Label(value="ArcGIS Online Password:"), self.form_agol_pw]),
            widgets.Box([widgets.Label(value="Portal URL:"), self.form_portal_url]),
            widgets.Box([widgets.Label(value="Portal Username:"), self.form_portal_username]),
            widgets.Box([widgets.Label(value="Portal Password:"), self.form_portal_pw]),
            self.form_connect_button
        ]

        self.form = widgets.Box(self.form_items, layout=widgets.Layout(display="flex", flex_flow="column", border="solid 2px", align_items="flex-start"))

        self.agol_username_value = None
        self.agol_pw_value = None
        self.portal_username_value = None
        self.portal_pw_value = None
        self.portal_url_value = None

        display(self.form)

    def submit_gis_details_clicked(self, b):
        self.agol_username_value = self.form_agol_username.value
        self.agol_pw_value = self.form_agol_pw.value
        self.portal_username_value = self.form_portal_username.value
        self.portal_pw_value = self.form_portal_pw.value
        self.portal_url_value = self.form_portal_url.value

    def get_agol_username_value(self):
        return self.agol_username_value

    def get_agol_pw_value(self):
        return self.agol_pw_value

    def get_form_portal_username(self):
        return self.portal_username_value

    def get_portal_pw_value(self):
        return self.portal_pw_value

    def get_portal_url_value(self):
        return self.portal_url_value

# Create an instance of the form
form = UserInputForm()


#### Connect to ArcGIS Enterprise and ArcGIS Online

In [ ]:

agol_proxy_username = form.get_agol_username_value()
agol_proxy_password = form.get_agol_pw_value()
portal_username = form.get_form_portal_username()
portal_password = form.get_portal_pw_value()
portal_url = form.get_portal_url_value()

# connect to arcgis online
gis_agol = GIS() # item is hosted publicly 
gis_ent = GIS(portal_url,portal_username,portal_password) # Enterprise 10.9.1 and above supported  - Note: you may need to add ",verify_cert=False" if you see certificate errors.

# get a token from ArcGIS enterprise - this will be used when we update item to provide credentials
ent_token = gis_ent._con.token

if ent_token:
    print("connected to enterprise, please proceed with processing...")



#### Functions

In [5]:
# Clone the item from ArcGIS Online to ArcGIS Enterprise
def clone_agol_item_to_enterprise(basemap_folder,gis_ent_conn,agol_item_to_clone):
    cloned_item = None
    try:
        # clone the item from agol    
        cloned_item_list = gis_ent_conn.content.clone_items(items=[agol_item_to_clone], folder=basemap_folder)
        # check to see if item was cloned and assign it to return variable
        if len(cloned_item_list) == 1:
            cloned_item = cloned_item_list[0]
           
    except Exception as e:
        print(f"  An error occurred cloning item: {e}")

    return cloned_item

In [6]:
# Update the item to use stored credentials
def update_service_creds(ent_item_to_update,agol_username,agol_password,enterprise_token):
    update_results = False
    try:
        update_results = ent_item_to_update.update(item_properties = {
                                           "url": ent_item_to_update.url,
                                           "serviceUsername": agol_username,
                                           "servicePassword": agol_password,
                                           "token": enterprise_token
                                           })
    except Exception as e:
        print(f"  An error occurred while updating item properties for the item: {e}")
    
    return update_results

In [7]:
# update the json to point to proxied source
# we will need to check to make sure the new enterprise url are changing to 
# matches the agol url - we just need to check the "end" of the map service
def update_resource_file(enterprise_item):
    update_results = False
    try:
        
        # get the name of the map service to use for checking 
        enterprise_map_search_index = enterprise_item.url.find('/rest/services/') + len('/rest/services/')
        enterprise_map_service_ends_with = enterprise_item.url[enterprise_map_search_index:]

        # get the styles/root.json  
        styles_json = enterprise_item.resources.get(file="styles/root.json")

        # update the values that reference arcgis online - repoint them to enterprise
        styles_json["sprite"] = "../sprites/sprite"
        styles_json["glyphs"] =  enterprise_item.url + "/resources/fonts/{fontstack}/{range}.pbf"
        for url_source in styles_json["sources"].items():
            if url_source[1]["type"] == "vector":
                if url_source[1]["url"].endswith(enterprise_map_service_ends_with):
                    url_source[1]["url"] = enterprise_item.url
                else:
                    print("  ERROR: vector basemap contains additional map url, this is not currently supported with this script")
                    print(f"  {url_source[1]['url']}")

        # update file with edits and get return success status
        update_results = enterprise_item.resources.update(folder_name = "styles", file_name ="root.json",text=styles_json)["success"]

    except Exception as e:
        print(f"  An error occurred while updating resource file of the item: {e}")    
        
    return update_results

#### Process Basemap List

In [ ]:
# create folder if it doesn't exist
folder_titles = [folder['title'] for folder in gis_ent.users.me.folders]
if basemap_folder not in folder_titles:
    gis_ent.content.folders.create(basemap_folder, owner= gis_ent.users.me.username)   
    print(f"Folder '{basemap_folder}' created successfully.")
    
# loop through the items in list, checking that each step of the process was completed
# if issues are encountered, error print messages are displayed and item is skipped.
for agol_item_id in agol_item_id_list:
    
    print("---------------------------")
    # --get agol item--
    # check to make sure we have an item
    agol_item = gis_agol.content.get(agol_item_id) 
    if not agol_item:
        print(f"  ERROR: could not find item: {agol_item_id} - skipping")
        continue # to next item in for loop   

    print(f"processing: {agol_item.type} - {agol_item.title}")
    
    
    # ---clone item from arcgis online to enterprise---
    # search for item in enterprise 
    # note the "typekeywords:source-"" format is a reliable way to find the clone of an item
    ent_item_list = gis_ent.content.search(query="typekeywords:source-" + agol_item_id,max_items=1)
    
    # if item exists skip it - delete any item you want to reclone 
    if len(ent_item_list) == 1: 
        print("  ERROR: item already exists - permanently delete item in portal manually first if you want to reclone")
        continue # to next item in for loop   

    ent_item = clone_agol_item_to_enterprise(basemap_folder,gis_ent,agol_item)

    
    # If item was cloned successfully, update the new item to use embedded credentials to agol
    if not ent_item:
        print("  ERROR: item was not cloned")
        continue # to next item in for loop   

    print(f"  cloned item from ArcGIS Online to ArcGIS Enterprise.")


    # ---update service credentials---
    update_service_creds_result = update_service_creds(ent_item,agol_proxy_username,agol_proxy_password,ent_token)
    
    # check if update to service credentials was successful - if it wasn't add message and continue to next item
    if not update_service_creds_result:
        print("  ERROR: service credentials could not be updated")
        continue # to next item in for loop  

    print(f"  updated service credentials")

    # ---update resource file for vector tile service---
    # we only need to update resource file it's a VectorTile service 
    if ent_item.type == "Vector Tile Service":
        update_resource_file_results = update_resource_file(ent_item)
    
        # check if update to resource file was successful 
        if update_resource_file_results:
            print ("  updated vector basemap resource file")
        else:
            print ("  ERROR: updating resource file")

    print(f"  item ready for use")

print("--processing complete--")    